# <MODEL_ID> — <MODEL_NAME> (exógenas)

**Objetivo:** forecasting de `Weekly_Sales` semanal por `Store` usando un modelo que incorpora variables exógenas.

## Supuesto experimental (oracle exog)
Se asume disponibilidad de todas las covariables exógenas durante el horizonte de predicción (escenario oracle).

## Outputs estándar
- `outputs/predictions/<modelo>_predictions.csv` con: `Store, Date, y_true, y_pred, model`
- `outputs/metrics/<modelo>_metrics_global.csv`
- `outputs/metrics/<modelo>_metrics_by_store.csv`
- `outputs/figures/<modelo>_plot_*.png`

In [1]:
# 0) Imports y configuración
from __future__ import annotations

import json
import sys
from pathlib import Path

import numpy as np
import pandas as pd

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.common import (
    compute_metrics,
    load_data,
    make_features,
    save_outputs,
    temporal_split,
)

MODEL_NAME = '<modelo>'  # e.g., 'sarimax_exog'
SEED = 42
np.random.seed(SEED)

DATA_PATH = PROJECT_ROOT / 'data' / 'Walmart_Sales.csv'
METADATA_PATH = PROJECT_ROOT / 'outputs' / 'metadata.json'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'

## 1) Cargar metadata (split + features)
Esto garantiza consistencia entre modelos.

In [2]:
metadata = json.loads(METADATA_PATH.read_text(encoding='utf-8'))
split = metadata['split']
feature_cols = metadata['features']
print('Split:', split)
print('N features:', len(feature_cols))

Split: {'train_start': '2010-02-05', 'train_end': '2012-07-06', 'val_start': '2012-07-13', 'val_end': '2012-08-31', 'test_start': '2012-09-07', 'test_end': '2012-10-26'}
N features: 16


## 2) Carga de datos + features
- Parseo/orden
- Construcción de lags/rolling (sin leakage)
- Exógenas alineadas por fecha

In [3]:
df = load_data(DATA_PATH)
df_feat, _ = make_features(df, add_calendar=True)

# Importante: para entrenar, debes decidir cómo tratar NaNs creados por lags/rolling
# Opción típica: descartar filas con NaNs en features (por store al inicio)
model_df = df_feat.dropna(subset=feature_cols + ['Weekly_Sales']).copy()
model_df.shape

(4095, 22)

## 3) Split temporal
Reutiliza exactamente el split definido en el notebook 00.

In [4]:
# Alternativa A (recomendado): recomputar split con los mismos parámetros del notebook 00
# (evita errores si el metadata se cambia manualmente).
train_df, val_df, test_df, split_cfg = temporal_split(df)

# Aplicar el split sobre model_df (ya sin NaNs por lags)
train = model_df[model_df['Date'].between(split_cfg.train_start, split_cfg.train_end)].copy()
val = model_df[model_df['Date'].between(split_cfg.val_start, split_cfg.val_end)].copy()
test = model_df[model_df['Date'].between(split_cfg.test_start, split_cfg.test_end)].copy()

print(len(train), len(val), len(test))

3375 360 360


## 4) Entrenamiento del modelo
Implementación específica por modelo (clásico por tienda o global multi-serie).

Requisito: usar exógenas en el modelo.

In [5]:
# TODO: implementar entrenamiento
# Debe producir predicciones para VAL y/o TEST (ideal: ambas).
# Estructura mínima esperada:
# - y_pred_test: np.ndarray o pd.Series alineada con test
#
# Por ahora, placeholder:
y_pred_test = np.full(shape=len(test), fill_value=test['Weekly_Sales'].mean())

## 5) Métricas (MAE, RMSE, sMAPE)
Se reporta:
- Global
- Por store

In [6]:
pred_df = pd.DataFrame({
    'Store': test['Store'].astype(int).values,
    'Date': test['Date'].values,
    'y_true': test['Weekly_Sales'].values,
    'y_pred': np.asarray(y_pred_test, dtype=float),
    'model': MODEL_NAME,
})

global_metrics = compute_metrics(pred_df['y_true'].values, pred_df['y_pred'].values)
metrics_global_df = pd.DataFrame([{'model': MODEL_NAME, **global_metrics}])

by_store = []
for store, g in pred_df.groupby('Store'):
    m = compute_metrics(g['y_true'].values, g['y_pred'].values)
    by_store.append({'model': MODEL_NAME, 'Store': int(store), **m})
metrics_by_store_df = pd.DataFrame(by_store).sort_values('Store')

metrics_global_df, metrics_by_store_df.head()

(      model            MAE           RMSE      sMAPE
 0  <modelo>  441124.924609  518987.216845  46.104407,
       model  Store           MAE          RMSE       sMAPE
 0  <modelo>      1  5.320871e+05  5.377143e+05   41.427339
 1  <modelo>      2  8.449641e+05  8.478403e+05   58.743041
 2  <modelo>      3  6.008675e+05  6.010464e+05   84.246824
 3  <modelo>      4  1.108559e+06  1.109581e+06   70.660465
 4  <modelo>      5  6.927589e+05  6.929573e+05  103.826028)

## 6) Guardado de outputs estándar

In [7]:
from src.common import save_outputs
paths = save_outputs(
    model_name=MODEL_NAME,
    predictions=pred_df,
    metrics_global=metrics_global_df,
    metrics_by_store=metrics_by_store_df,
    output_dir=OUTPUTS_DIR,
)
paths

{'predictions': '/mnt/custom-file-systems/s3/shared/TFMAXEL/outputs/predictions/<modelo>_predictions.csv',
 'metrics_global': '/mnt/custom-file-systems/s3/shared/TFMAXEL/outputs/metrics/<modelo>_metrics_global.csv',
 'metrics_by_store': '/mnt/custom-file-systems/s3/shared/TFMAXEL/outputs/metrics/<modelo>_metrics_by_store.csv'}

## 7) Figuras
Recomendado para la memoria:
- 3 tiendas: serie real vs predicha en test
- Distribución del error (`y_true - y_pred`)

(Cada notebook de modelo debe guardar sus figuras en `outputs/figures/`).

In [8]:
# TODO: implementar plots por modelo (matplotlib/seaborn)
# Guardar como PNG en outputs/figures/
pass